## Packages Used

In [68]:
import pandas as pd
import numpy as np
from datetime import datetime, timedelta, date
from dateutil.relativedelta import relativedelta


### CSV File Import

In [69]:
name = 'Workout Progress Data - master.csv'
workout_data = pd.read_csv(name)

## Exploring Data

In [70]:
# Printing the first 5 rows
workout_data.head()

,Date,Exercise,Muscle_group,Sets,Reps,Weight,Duration,Note
0,11/24/22,Pull Up,Back,1.0,6.0,BW,NaN,NaN
1,11/24/22,Pull Up,Back,2.0,5.0,BW,NaN,NaN
2,11/24/22,Dips,Triceps,1.0,14.0,BW,NaN,NaN
3,11/24/22,Dips,Triceps,2.0,11.0,BW,NaN,NaN
4,11/24/22,Dips,Triceps,3.0,10.0,BW,NaN,NaN


In [71]:
# Printing the last 5 rows
workout_data.tail()

,Date,Exercise,Muscle_group,Sets,Reps,Weight,Duration,Note
172,11/27/22,Push Up,Chest,2.0,10.0,BW,NaN,Parallette Push-Up
173,11/27/22,Push Up,Chest,3.0,10.0,BW,NaN,Parallette Push-Up
174,11/27/22,Ab Crunch,Abs,1.0,20.0,BW,NaN,NaN
175,11/27/22,Ab Crunch,Abs,2.0,15.0,BW,NaN,NaN
176,11/27/22,Flutter Kicks,Abs,1.0,20.0,BW,NaN,NaN


In [72]:
# Data shape
workout_data.shape

(177, 8)

In [73]:
# Printing data columns
workout_data.columns

Index(['Date', 'Exercise', 'Muscle_group', 'Sets', 'Reps', 'Weight',
       'Duration', 'Note'],
      dtype='object')

In [74]:
# Dara more information
workout_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 177 entries, 0 to 176
Data columns (total 8 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   Date          177 non-null    object 
 1   Exercise      177 non-null    object 
 2   Muscle_group  177 non-null    object 
 3   Sets          175 non-null    float64
 4   Reps          175 non-null    float64
 5   Weight        177 non-null    object 
 6   Duration      0 non-null      float64
 7   Note          3 non-null      object 
dtypes: float64(3), object(5)
memory usage: 11.2+ KB


## Fields

In [75]:
# Date Parameters -- Calculations
today = date.today()
startOfWeek = today - relativedelta(days=today.weekday())
endOfWeek = startOfWeek + relativedelta(days=6)
firstDayOfMonth = today.replace(day=1)
nextMonthDate = today + relativedelta(months=1, day=1)

# Formatting into string
today = today.strftime('%m-%d-%Y')
startOfWeek = startOfWeek.strftime('%m-%d-%Y')
end_of_week = endOfWeek.strftime('%m-%d-%Y')
firstDayOfMonth = firstDayOfMonth.strftime('%m-%d-%Y')
nextMonthDate = nextMonthDate.strftime('%m-%d-%Y')

# Output
message = f""""
Today: {str(today)} \n
Week Start: {str(startOfWeek)}
Week End: {str(end_of_week)} \n
Month Start: {str(firstDayOfMonth)}
Next Month Start: {str(nextMonthDate)}
"""
print(message)

"
Today: 11-28-2022 

Week Start: 11-28-2022
Week End: 12-04-2022 

Month Start: 11-01-2022
Next Month Start: 12-01-2022



### Sorting Data

In [76]:
workout_data = workout_data.sort_values(by=['Date','Exercise','Sets'])

In [77]:
# Unique days count
unique_days_list = pd.DataFrame(workout_data['Date'].unique(), columns=['Date'])
print(f"Count of unique days in all csv: {len(unique_days_list)}")


Count of unique days in all csv: 5


---

### Current Month Data
Querying all data related to the current month

In [78]:
# current_month_data = workout_data[workout_data['Date'] >= firstDayOfMonth]
# current_month_data

current_month_data = workout_data.query(" Date >= @firstDayOfMonth ")

In [79]:
current_month_data

,Date,Exercise,Muscle_group,Sets,Reps,Weight,Duration,Note
56,11/23/22,Bicep Curl,Biceps,1.0,10.0,15,NaN,NaN
57,11/23/22,Bicep Curl,Biceps,2.0,10.0,20,NaN,NaN
58,11/23/22,Bicep Curl,Biceps,3.0,10.0,20,NaN,NaN
59,11/23/22,Bicep Curl,Biceps,4.0,10.0,15,NaN,NaN
60,11/23/22,Bicep Curl,Biceps,5.0,10.0,20,NaN,NaN
...,...,...,...,...,...,...,...,...
143,11/27/22,Pull Up,Back,1.0,8.0,BW,NaN,NaN
144,11/27/22,Pull Up,Back,2.0,5.0,BW,NaN,NaN
171,11/27/22,Push Up,Chest,1.0,10.0,BW,NaN,Parallette Push-Up
172,11/27/22,Push Up,Chest,2.0,10.0,BW,NaN,Parallette Push-Up


In [80]:
current_month_data["Date"].unique()

array(['11/23/22', '11/24/22', '11/25/22', '11/26/22', '11/27/22'],
      dtype=object)

In [81]:
current_month_data

,Date,Exercise,Muscle_group,Sets,Reps,Weight,Duration,Note
56,11/23/22,Bicep Curl,Biceps,1.0,10.0,15,NaN,NaN
57,11/23/22,Bicep Curl,Biceps,2.0,10.0,20,NaN,NaN
58,11/23/22,Bicep Curl,Biceps,3.0,10.0,20,NaN,NaN
59,11/23/22,Bicep Curl,Biceps,4.0,10.0,15,NaN,NaN
60,11/23/22,Bicep Curl,Biceps,5.0,10.0,20,NaN,NaN
...,...,...,...,...,...,...,...,...
143,11/27/22,Pull Up,Back,1.0,8.0,BW,NaN,NaN
144,11/27/22,Pull Up,Back,2.0,5.0,BW,NaN,NaN
171,11/27/22,Push Up,Chest,1.0,10.0,BW,NaN,Parallette Push-Up
172,11/27/22,Push Up,Chest,2.0,10.0,BW,NaN,Parallette Push-Up


In [82]:
# For loop over list of unique dates in curentMonthData dates's column
# Counting each loop as a Record Day
daysCount = 0
for date in current_month_data[current_month_data['Date'] >= firstDayOfMonth]['Date'].unique():
    print(date)
    daysCount += 1

11/23/22
11/24/22
11/25/22
11/26/22
11/27/22


In [83]:
lastRecordDate = current_month_data['Date'].unique()[-1]
lastRecordData = current_month_data[current_month_data['Date'] == lastRecordDate]

lastRecordData

,Date,Exercise,Muscle_group,Sets,Reps,Weight,Duration,Note
174,11/27/22,Ab Crunch,Abs,1.0,20.0,BW,NaN,NaN
175,11/27/22,Ab Crunch,Abs,2.0,15.0,BW,NaN,NaN
142,11/27/22,Bodyweight,Weight,NaN,NaN,127.1,NaN,NaN
145,11/27/22,Chin Up,Biceps,1.0,5.0,BW,NaN,NaN
146,11/27/22,Dips,Triceps,1.0,10.0,BW,NaN,NaN
147,11/27/22,Dips,Triceps,2.0,10.0,BW,NaN,NaN
148,11/27/22,Dips,Triceps,3.0,10.0,BW,NaN,NaN
168,11/27/22,Dumbbell Overhead Triceps Extension,Triceps,1.0,10.0,35,NaN,NaN
169,11/27/22,Dumbbell Overhead Triceps Extension,Triceps,2.0,10.0,35,NaN,NaN
170,11/27/22,Dumbbell Overhead Triceps Extension,Triceps,3.0,10.0,35,NaN,NaN


In [84]:
# what muscle did we work out last
for unique_muscle in lastRecordData['Muscle_group'].unique():
    print(unique_muscle)

Abs
Weight
Biceps
Triceps
Shoulders
Back
Chest
